In [1]:
import pandas as pd
import numpy as np
#import tensorflow as tf
import re
import time
from afinn import Afinn
#tf.__version__

In [2]:
# Load the data
emotion_lines = open('dialogues_emotion.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open('dialogues_text.txt', encoding='utf-8', errors='ignore').read().split('\n')
act_lines = open('dialogues_act.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [3]:
# The sentences that we will be using to train our model.
#lines[:10]

In [4]:
# The sentences' ids, which will be processed to become our input and target data.
#conv_lines[:10]

In [7]:

# Create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]


NameError: name 'lines' is not defined

In [8]:
# Create a list of all of the conversations' lines' ids.
convs = [ ]
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

In [9]:


convs[:10]

[['hekitchenstinks.__eou__Illthrowoutthegarbage.__eou_'],
 ['oDick',
  'howaboutgettingsomecoffeefortonight?__eou__Coffee?Idon’thonestlylikethatkindofstuff.__eou__Comeon',
  'youcanatleasttryalittle',
  'besidesyourcigarette.__eou__What’swrongwiththat?CigaretteisthethingIgocrazyfor.__eou__Notforme',
  'Dick.__eou_'],
 ['rethingsstillgoingbadlywithyourhouseguest?__eou__Gettingworse.Nowhe’seatingmeoutofhouseandhome.I’Vetriedtalkingtohimbutitallgoesinoneearandouttheother.Hemakeshimselfathome',
  'whichisfine.ButwhatreallygetsmeisthatyesterdayhewalkedintothelivingroomintherawandIhadcompanyover!Thatwasthelaststraw.__eou__Leo',
  'Ireallythinkyou’rebeatingaroundthebushwiththisguy.Iknowheusedtobeyourbestfriendincollege',
  'butIreallythinkit’stimetolaydownthelaw.__eou__You’reright.Everythingisprobablygoingtocometoaheadtonight.I’llkeepyouinformed.__eou_'],
 ['ouldyoumindwaitingawhile?__eou__Well',
  'howlongwillitbe?__eou__Imnotsure.ButIllgetatablereadyasfastasIcan.__eou__OK.Wellwait.__eou_'],

In [35]:
# Load the data
emotion_lines = open('dialogues_emotion.txt', encoding='utf-8', errors='ignore').read().split('\n')
act_lines = open('dialogues_act.txt', encoding='utf-8', errors='ignore').read().split('\n')
text_lines = open('dialogues_text.txt', encoding='utf-8', errors='ignore').read().split('\n')

emotions =[] 
for i in range(len(emotion_lines)):
    emotions.append(emotion_lines[i].split(" "))

act = []
for i in range(len(act_lines)):
    act.append(act_lines[i].split(" "))

corpus = [] 
for i in range(len(text_lines)):
    corpus.append(text_lines[i].split("__eou__"))

In [36]:
emotions[0]

['2', '0', '']

In [37]:
corpus[0]

['The kitchen stinks . ', " I'll throw out the garbage . ", '']

In [61]:
new_questions = []
new_answers = []
afinn = Afinn(language='en')  

len(corpus)-1
for j in range(len(corpus)-1):
    for i in range(len(corpus[j])-2):
        if(emotions[j][i] == '4'):
            if afinn.score(corpus[j][i+1]) >= 0:
                new_questions.append(corpus[j][i])
                new_answers.append(corpus[j][i+1])
                

print(len(new_questions))   
print(len(new_answers))   

8636
8636


In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

In [63]:
#nltk.download('punkt') # first-time use only
#nltk.download('wordnet') # first-time use only


In [64]:
import string # to process standard python strings
from sklearn.metrics.pairwise import cosine_similarity


In [65]:
#sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 

In [66]:
# Preprocessing
lemmer = nltk.WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


In [67]:
"""
def generate_response(questions,answers,user_input):
    
    questions.append(user_input)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize)
    tfidf = TfidfVec.fit_transform(questions)

    
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-1]
    
    if(req_tfidf==0):
            robo_response = "I am sorry! I don't understand you"
    else:
            robo_response = answers[idx]
            
    return robo_response
"""

'\ndef generate_response(questions,answers,user_input):\n    \n    questions.append(user_input)\n    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize)\n    tfidf = TfidfVec.fit_transform(questions)\n\n    \n    vals = cosine_similarity(tfidf[-1], tfidf)\n    idx=vals.argsort()[0][-2]\n    \n    flat = vals.flatten()\n    flat.sort()\n    req_tfidf = flat[-1]\n    \n    if(req_tfidf==0):\n            robo_response = "I am sorry! I don\'t understand you"\n    else:\n            robo_response = answers[idx]\n            \n    return robo_response\n'

In [72]:
def generate_response(questions,answers,user_input):
    
    #print(len(questions))
    #print(len(answers))
    questions.append(user_input)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize)
    tfidf = TfidfVec.fit_transform(questions)

    
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-1]
    
    """
    question_response = [questions[idx], questions[vals.argsort()[0][-3]],questions[vals.argsort()[0][-4]],
                    questions[vals.argsort()[0][-5]],questions[vals.argsort()[0][-6]]]
    robo_response = [answers[idx], answers[vals.argsort()[0][-3]],answers[vals.argsort()[0][-4]],
                    answers[vals.argsort()[0][-5]],answers[vals.argsort()[0][-6]]]
    """
    if(req_tfidf==0):
            robo_response = "I am sorry! I don't understand you"
    else:
            #print(idx, "idex")
            robo_response = answers[idx]
    
    answers.append(robo_response)
    #return question_response, robo_response
    return robo_response

In [ ]:
quit = False

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey")
GREETING_RESPONSES = "Hello Cutie"

while quit == False:
    print("User: ")
    user_input = input()
    print("Robo: ")
    if(user_input in GREETING_INPUTS):
        print(GREETING_RESPONSES)
    elif(user_input!='bye'):
        print(generate_response(new_questions,new_answers,user_input))
    else:
        print("bye")
        quit = True


User: 
HI
Robo: 
 Yes . I moved here last week . 
User: 


In [ ]:
import json

In [ ]:
with open('frames.json', 'r') as myfile:
    data=myfile.read()
    
# parse file
obj = json.loads(data)

# show values
print(obj[100].get("text"))

    

In [ ]:
print(type(obj))